In [2]:
import pysam
import collections
import random
import pandas as pd
import numpy as np
%load_ext rpy2.ipython


/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: During startup - 
  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: Warning messages:

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: 1: package ‘methods’ was built under R version 3.2.4 

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: 2: package ‘datasets’ was built under R version 3.2.4 

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: 3: package ‘utils’ was built under R version 3.2.4 

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarnin

In [20]:
infile = "/ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/Nxf1-GFP-R2.bam"

This notebook contains test code to examine whether indel mutations in UMIs present a substainal effect. To test this, we first recognise that a one base pair indel will suck a base of the actual genomic sequencing into the UMI, shifting the mapping position of the by one base, and including a genome determined base as the final base of the UMI.

To test this we will parse the bam file and fill a dictionary: key="[contig][position]", value= counter of UMIs for each position. Then we'll parse the dictionary keys and look to see whether the positions +1 exist in the dictionary keys. If they do, we'll compare the umi profiles between the postions. 

First we will filter for UMIs that have the genomic base in the final UMI position. Then we will check for each possible one bp deletion in the reference UMI and see if that UMI exists in the +1 position. We will calculate what % of the UMIs at the reference position exist as deletion versions at +1 and what % of reads at +1 are deletion versions releative to the reference base.

This will then be compared to a random sample

In [29]:
#Test set
umis1 = collections.Counter()
umis2 = collections.Counter()
umis1["ATAGT"] = 100
umis1["AGTAG"] = 10
umis1["GCGAT"] = 150

umis2["ATGTA"] = 2
umis2["GACAG"] = 1000
umis2["CCAGA"] = 20

print umis1
print umis2

Counter({'GCGAT': 150, 'ATAGT': 100, 'AGTAG': 10})
Counter({'GACAG': 1000, 'CCAGA': 20, 'ATGTA': 2})


A function to calculate the fraction of the reference and plus 1 base UMIs that are part of deletion pairs

In [7]:
def getDelFraction(counter1, counter2, genomic_base):
    ''' for two counters with string keys,
    return the fraction of bases at counter2 that could be deletions cf conuter1'''
    
    umis1 = counter1.keys()
    umis2 = counter2.keys()
    
    found1 = set()
    found2 = set()
    
    filtered_set2 = set([umi[:-1] for umi in umis2 if umi[-1] == genomic_base])
    for umi in umis1:
        for i in range(len(umi)):
            del_umi = umi[:i] + umi[i+1:]
            if del_umi in filtered_set2:
                found1.add(umi)
                found2.add(del_umi+genomic_base)
            
    return {"as_fraction_of_ref": float(len(found1))/len(umis1),
            "as_fraction_of_plus1": float(len(found2))/len(umis2),
            "weighted_fraction_of_ref": float(sum([counter1[umi] for umi in found1]))/sum(counter1.values()),
            "weighted_fraction_of_plus1": float(sum([counter2[umi] for umi in found2]))/sum(counter2.values())}
getDelFraction(umis1, umis2, 'A')

{'as_fraction_of_plus1': 0.3333333333333333,
 'as_fraction_of_ref': 0.3333333333333333,
 'weighted_fraction_of_plus1': 0.0019569471624266144,
 'weighted_fraction_of_ref': 0.38461538461538464}

now a function to parse a bam and create the dictionary.

Because whether we care about +1 or -1 and whether we want to rev comp the sequence or not, we will deal with plus and minus strands seperately. 

In [68]:
def parse_samfile(infile):
    '''Parses a bamfile and returns three dictionaries, the first is a dictionary of counters
    with the count of each umi at each bases on each contig, the second is the first bases matched
    reads at that position, and the third is the distribution of the UMIs in the file'''
    insam = pysam.Samfile(infile, "rb")

    umi_pos = collections.defaultdict(lambda:collections.defaultdict(lambda: collections.Counter()))
    umi_dist=collections.Counter()
    genomic_bases = collections.defaultdict(lambda: collections.defaultdict(str))
    inreads = insam.fetch()
    for read in inreads:
        if read.is_unmapped:
            continue

        if read.mate_is_unmapped and paired:
            continue

        if read.is_read2:
            continue

        is_spliced = False

        if read.is_reverse:
            continue
        
        else:
            pos = read.pos
            if read.cigar[0][0] == 4:
                pos = pos - read.cigar[0][1]
            start = pos

            if ('N' in read.cigarstring or
                (read.cigar[-1][0] == 4 and
                 read.cigar[-1][1] > soft_clip_threshold)):
                is_spliced = True

        umi = read.qname.split("_")[-1]
        chrom = insam.get_reference_name(read.tid)
        umi_pos[chrom][pos][umi] += 1
        umi_dist[umi] += 1
        
        if read.cigar[0][0] == 0:
            genomic_bases[chrom][pos] = read.query_sequence[0]
                
    return umi_pos, umi_dist, genomic_bases

Start with an undeduped sample. 

In [69]:
undeduped_umi_pos, undeduped_umi_dist, undeduped_bases = parse_samfile(infile)

In [48]:
n = 0
print len(undeduped_umi_pos)
for k,v in undeduped_umi_pos.iteritems():
    n += len(v)
print n

22
370674


Now we need to go through each pair of adjecent positions and calculate the fraction of the UMIs at position +1 that could be explained as deletions of UMIs at the reference position. Then randomise the UMIs at the +1 position and do the same. 

In [65]:
def randomise_position(umi_counter, umi_dist):
    '''Takes a counter of UMI frequencies and create a randomised distribution by replacing the UMIs 
    in the input which UMIs sampled from the genomewide distribution'''
    
    return {umi: count for umi, count in 
            zip(np.random.choice(umi_dist["index"], 
                                 size=len(umi_counter.keys()),
                                 replace=False,
                                 p=umi_dist["freq"]),
                                 umi_counter.values())}

In [74]:
undeduped_umi_dist = pd.Series(undeduped_umi_dist, name="count").reset_index()
undeduped_umi_dist["freq"] = undeduped_umi_dist["count"] / undeduped_umi_dist["count"].sum()

In [72]:
def calculate_deletion_rate(umi_pos, umi_dist, genomic_bases):
    '''Find positions where position+1 also has UMIs and calculate the deletion rate,
    in comparision to randomised UMIs'''
    skipped = 0
    results_accumulator = []
    random_accumulator = []
    for contig in umi_pos:
        for position in umi_pos[contig]:
            if position+1 in umi_pos[contig]:
                genomic_base = genomic_bases[contig][position]
                if genomic_base == str():
                    skipped += 1
                    continue
                this =  umi_pos[contig][position]
                other = umi_pos[contig][position + 1]
                

                rand_other = randomise_position(other, umi_dist)
                results_accumulator.append(getDelFraction(this, other, genomic_base))
                random_accumulator.append(getDelFraction(this, rand_other, genomic_base))
            
    results_frame = pd.DataFrame(results_accumulator)
    random_accumulator = pd.DataFrame(random_accumulator)

    results_frame["random"] = 0
    random_accumulator["random"] = 1

    results_frame = pd.concat([results_frame, random_accumulator])
    print skipped
    return results_frame



In [75]:
undeduped_deletion_rates = calculate_deletion_rate(undeduped_umi_pos, undeduped_umi_dist, undeduped_bases)

0


Having calculated the deledtion rates, lets look at the distribution, for both randomised and non-randomised

In [85]:
undeduped_deletion_rates.groupby("random").describe()

as_fraction_of_plus1  as_fraction_of_ref  \
random                                                   
0      count          35998.000000        35998.000000   
       mean               0.050737            0.072235   
       std                0.135960            0.187270   
       min                0.000000            0.000000   
       25%                0.000000            0.000000   
       50%                0.000000            0.000000   
       75%                0.000000            0.000000   
       max                1.000000            1.000000   
1      count          35998.000000        35998.000000   
       mean               0.039568            0.065143   
       std                0.100434            0.173073   
       min                0.000000            0.000000   
       25%                0.000000            0.000000   
       50%                0.000000            0.000000   
       75%                0.000000            0.000000   
       max                1.000000            1.000000   

              weighted_fraction_of_plus1  weighted_fraction_of_ref  
random                                                              
0      count                35998.000000              35998.000000  
       mean                     0.051454                  0.072904  
       std                      0.143054                  0.192871  
       min                      0.000000                  0.000000  
       25%                      0.000000                  0.000000  
       50%                      0.000000                  0.000000  
       75%                      0.000000                  0.000000  
       max                      1.000000                  1.000000  
1      count                35998.000000              35998.000000  
       mean                     0.039544                  0.066186  
       std                      0.107504                  0.180149  
       min                      0.000000                  0.000000  
       25%                      0.000000                  0.000000  
       50%                      0.000000                  0.000000  
       75%                      0.000000                  0.000000  
       max                      1.000000                  1.000000

There is an enrichment compared to the randomisations. Small but clearly present. The overlap is zero in the vast majority of cases. The important measure is the UMIs as a fractin of the +1 position - these are UMIs that are not real. The means are different, but the overlap is zero in the vast majoirty of cases. How many cases are they non-zero?

In [77]:
undeduped_deletion_rates.groupby("random").apply(lambda x: (x>0).sum())

,as_fraction_of_plus1,as_fraction_of_ref,weighted_fraction_of_plus1,weighted_fraction_of_ref,random
random,,,,,
0,8593,8593,8593,8593,0
1,8276,8276,8276,8276,35998


Its a very small difference at this level. Some 300 positions or so. The real imporant descision is whether or not this enrichment is still present after deduplication. 

Compare to the deduped files:

In [80]:
deduped_umi_pos, deduped_umi_dist, deduped_bases = parse_samfile(
    "/ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/dedup_directional-adjacency.dir/Nxf1-GFP-R2.bam")
deduped_umi_dist = pd.Series(deduped_umi_dist, name="count").reset_index()
deduped_umi_dist["freq"] = deduped_umi_dist["count"] / deduped_umi_dist["count"].sum()
deduped_deletion_rates = calculate_deletion_rate(deduped_umi_pos, deduped_umi_dist, deduped_bases)

0


In [83]:
deduped_deletion_rates.groupby("random").mean()

,as_fraction_of_plus1,as_fraction_of_ref,weighted_fraction_of_plus1,weighted_fraction_of_ref
random,,,,
0,0.045836,0.053993,0.045836,0.053993
1,0.035691,0.046084,0.035691,0.046084


In [82]:
deduped_deletion_rates.groupby("random").apply(lambda x: (x>0).sum())

,as_fraction_of_plus1,as_fraction_of_ref,weighted_fraction_of_plus1,weighted_fraction_of_ref,random
random,,,,,
0,8147,8147,8147,8147,0
1,7708,7708,7708,7708,35998


The enrichment is smaller, but not by much. The biggest difference is that the there is a smaller number of overlapping positions in the randomised positions.

How much are the enrichments:

In [87]:
undeduped_means = undeduped_deletion_rates.groupby("random").mean()
undeduped_means.loc[0]/undeduped_means.loc[1]

as_fraction_of_plus1          1.282277
as_fraction_of_ref            1.108869
weighted_fraction_of_plus1    1.301189
weighted_fraction_of_ref      1.101507
dtype: float64

In [88]:
deduped_means = deduped_deletion_rates.groupby("random").mean()
deduped_means.loc[0]/deduped_means.loc[1]

as_fraction_of_plus1          1.284248
as_fraction_of_ref            1.171609
weighted_fraction_of_plus1    1.284248
weighted_fraction_of_ref      1.171609
dtype: float64

What are these enrichments compared to the enrichments caused by edit_disatance enrichments?

In [90]:
edit_distance = pd.read_csv(
    "/ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/dedup_directional-adjacency.dir/Nxf1-GFP-R2_edit_distance.tsv", sep="\t")
edit_distance = edit_distance.set_index("edit_distance")
edit_distance


,directional-adjacency,directional-adjacency_null,unique,unique_null
edit_distance,,,,
Single_UMI,676164,676164,661329,661329
0,0,31,0,37
1,137,388,14791,592
2,3312,2311,3540,3511
3,7697,7980,8181,11627
4,43520,43954,43709,50401
5,8595,8597,7875,11928
6,0,0,0,0


In [94]:
edit_distance_fractions = edit_distance.drop("Single_UMI", axis=0).apply(lambda x: x/sum(x))

In [95]:
edit_distance_fractions["unique"]/edit_distance_fractions["unique_null"]

edit_distance
0     0.000000
1    24.984797
2     1.008260
3     0.703621
4     0.867225
5     0.660211
6          NaN
dtype: float64

So the substitution errors are a 25x enrichment, rather than a 1.3 fold enrichment for the deletions. 